In [ ]:
import pandas as pd
import sqlite3
import time
from datetime import datetime

In [ ]:
def standardize_column_name(column_name):
    return column_name.strip().lower().replace(' ', '_')

In [ ]:
def excel_to_sqlite_delta(excel_file: str, db_file: str, table_name: str, unique_key: str):
  """
  Reads an Excel file and updates an SQLite database with new records only.

  :param excel_file: Path to the Excel file.
  :param db_file: Path to the SQLite database file.
  :param table_name: Name of the table to store data in.
  :param unique_key: Column name that serves as a unique identifier.
  """
  try:
      # Load Excel file into DataFrame
      df = pd.read_excel(excel_file)
      df.columns = df.columns.map(standardize_column_name)

      # Connect to SQLite database
      conn = sqlite3.connect(db_file)
      cursor = conn.cursor()

      # Create table if not exists
      df.head(0).to_sql(table_name, conn, if_exists='append', index=False)

      # Fetch existing unique keys
      cursor.execute(f"SELECT {unique_key} FROM {table_name}")
      existing_keys = set(row[0] for row in cursor.fetchall())

      # Filter out already existing records
      new_data = df[~df[unique_key].isin(existing_keys)]

      if not new_data.empty:
          new_data.to_sql(table_name, conn, if_exists='append', index=False)
          print(f"Added {len(new_data)} new records to {table_name}")
      else:
          print("No new records to add.")
  except Exception as e:
      print(f"Error updating database: {e}")
  finally:
      conn.close()

In [ ]:
def query_sqlite(db_file: str, query: str):
  """
  Queries an SQLite database and returns the results.
  """
  try:
      conn = sqlite3.connect(db_file)
      df = pd.read_sql_query(query, conn)
      return df
  except Exception as e:
      print(f"Error executing query: {e}")
      return None
  finally:
      conn.close()

In [ ]:
def schedule_updates(excel_file: str, db_file: str, table_name: str, unique_key: str):
  """
  Runs the update process once and subsequently every hour, continuously.
  """
  sleep_time = 3600
  while True:
      try:
          time_now = datetime.now()
          print(f"Running update at {time_now.strftime('%Y-%m-%d %H:%M:%S')}")
          excel_to_sqlite_delta(excel_file, db_file, table_name, unique_key)
      except Exception as e:
          print(f"Error in scheduled update: {e}")
      finally:
          time.sleep(sleep_time)

In [ ]:
excel_file = '../data/orders_table.xlsx'
db_file = '../data/chatbot.db'
table_name = 'orders'
unique_key = 'Order ID'
unique_key = standardize_column_name(unique_key)

In [ ]:
schedule_updates(excel_file, db_file, table_name, unique_key)

Running update at 2025-03-06 18:01:08
No new records to add.


KeyboardInterrupt: 

In [ ]:
# Query data
query = f"SELECT * FROM {table_name} LIMIT 5;"
result_df = query_sqlite(db_file, query)
print(result_df)

   order_id customer_name product_category      product_name  size  gender  \
0     41331      John Doe     Casual Shoes   Canvas Sneakers    10    Male   
1     44335   Sarah Smith     Formal Shoes    Leather Oxford     8  Female   
2     93001  Mike Johnson          Sandals  Beach Flip-Flops    11    Male   
3     41007    Lisa Brown         Slippers     Home Slippers     6  Female   
4     43033    Alex Green     Casual Shoes  Running Sneakers     9  Unisex   

   quantity  price_(usd)           order_date      status payment_method  \
0         1        49.99  2024-12-28 00:00:00     Shipped    Credit Card   
1         2       129.99  2025-01-03 00:00:00  Processing         PayPal   
2         1        19.99  2025-01-04 00:00:00   Delivered     Debit Card   
3         1        14.99  2025-01-07 00:00:00  Processing    Credit Card   
4         1        59.99  2025-01-14 00:00:00     Shipped         PayPal   

  shipping_address final_sale  
0          NY, USA         No  
1         